<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Upload and Predict</title>
</head>
<body>
    <h1>Upload and Predict</h1>
    <form method="POST" enctype="multipart/form-data">
        <input type="file" name="image" accept="image/*" required>
        <button type="submit">Predict</button>
    </form>
    {% if image_path %}
    <img src="{{ image_path }}" alt="Uploaded Image" style="max-width: 400px">
    <p>{{ result }}</p>
    {% endif %}
</body>
</html>

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
import numpy as np
import cv2
import matplotlib.pyplot as plt

# Загрузка и предобработка набора данных MNIST
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Создание модели нейронной сети
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
])

# Компиляция модели
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Обучение модели
model.fit(x_train[..., np.newaxis], y_train, epochs=5)

# Функция для загрузки и обработки фото для распознавания
def process_image(file_path):
    img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (28, 28))
    img = np.array(img) / 255.0
    return img.reshape(1, 28, 28, 1)

# Загрузка изображения для распознавания и вывод результата
file_path = '3.png'  # Укажите путь к вашему изображению
input_image = process_image(file_path)
prediction = model.predict(input_image)
predicted_label = np.argmax(prediction)
print(f'Predicted label: {predicted_label}')

# Отображение изображения для визуальной проверки
plt.imshow(input_image.reshape(28, 28), cmap='gray')
plt.show()